In [ ]:
%pip install hana_ml/hana_ml-2.17.23061500-py3-none-any.whl
%pip install shapely
%pip install jinja2
%pip install requests
%pip install cryptography


In [2]:
import hana_ml
print(hana_ml.__version__)

2.17.23061500


In [3]:
import hana_ml
from hana_ml import dataframe
from hana_ml.dataframe import create_dataframe_from_pandas, create_dataframe_from_shapefile
from hana_ml.algorithms.pal.tsa.additive_model_forecast import AdditiveModelForecast
from hana_ml.algorithms.pal import metrics
print(hana_ml.__version__)

2.17.23061500


In [4]:
host = '4b9b5603-72ee-4b9a-856e-0ccefffb3148.hana.prod-eu10.hanacloud.ondemand.com'
port = 443
user = 'DEMO_USER'

In [5]:
conn= dataframe.ConnectionContext(address=host, port=port, user=user, 
                               encrypt='true' ,sslValidateCertificate='false')
print(conn.hana_version())


4.00.000.00.1684836416 (fa/CE2023.4)


In [6]:
test_gt_rnk_hdf = conn.table("TEST_GT_RNK_2209", schema="TECHED22")
print(test_gt_rnk_hdf.count())
test_rnk_hdf = conn.table("TEST_RNK_2209", schema="TECHED22")
print(test_rnk_hdf.count())
train_rnk_hdf = conn.table("TRAIN_RNK_2209", schema="TECHED22")
print(train_rnk_hdf.count())

17909
17908
354578


In [7]:
from hana_ml.algorithms.pal.tsa.additive_model_forecast import AdditiveModelForecast

In [8]:
conn.sql_tracer.enable_sql_trace(True)
conn.sql_tracer.enable_trace_history(True)

In [9]:
# create hoildays table, required but kept empty for now
holiday_data_hdf = conn.sql('select * from "TECHED22"."PAL_ADDITIVE_MODEL_ANALYSIS_HOLIDAY"')

In [10]:
amf = AdditiveModelForecast(massive=True,growth='linear',
                                changepoint_prior_scale=0.06,
                                weekly_seasonality='True',
                                daily_seasonality='True'
                                )

In [11]:
amf.fit(data=train_rnk_hdf, key="PRICE_AT_TIMESTAMP", group_key="STATION_UUID", holiday=holiday_data_hdf)

INFO:hana_ml.ml_base:Executing SQL: DO
BEGIN
DECLARE group_id NVARCHAR(5000) ARRAY;
DECLARE param_name VARCHAR(5000) ARRAY;
DECLARE int_value INTEGER ARRAY;
DECLARE double_value DOUBLE ARRAY;
DECLARE string_value VARCHAR(5000) ARRAY;
group_id[1] := N'PAL_MASSIVE_PROCESSING_SPECIAL_GROUP_ID';
param_name[1] := N'GROWTH';
int_value[1] := NULL;
double_value[1] := NULL;
string_value[1] := N'linear';
group_id[2] := N'PAL_MASSIVE_PROCESSING_SPECIAL_GROUP_ID';
param_name[2] := N'WEEKLY_SEASONALITY';
int_value[2] := 1;
double_value[2] := NULL;
string_value[2] := NULL;
group_id[3] := N'PAL_MASSIVE_PROCESSING_SPECIAL_GROUP_ID';
param_name[3] := N'DAILY_SEASONALITY';
int_value[3] := 1;
double_value[3] := NULL;
string_value[3] := NULL;
group_id[4] := N'PAL_MASSIVE_PROCESSING_SPECIAL_GROUP_ID';
param_name[4] := N'CHANGEPOINT_PRIOR_SCALE';
int_value[4] := NULL;
double_value[4] := 0.06;
string_value[4] := NULL;
params = UNNEST(:group_id, :param_name, :int_value, :double_value, :string_value);
in_0 = S

In [12]:
print(conn.last_execute_statement)

DO
BEGIN
DECLARE group_id NVARCHAR(5000) ARRAY;
DECLARE param_name VARCHAR(5000) ARRAY;
DECLARE int_value INTEGER ARRAY;
DECLARE double_value DOUBLE ARRAY;
DECLARE string_value VARCHAR(5000) ARRAY;
group_id[1] := N'PAL_MASSIVE_PROCESSING_SPECIAL_GROUP_ID';
param_name[1] := N'GROWTH';
int_value[1] := NULL;
double_value[1] := NULL;
string_value[1] := N'linear';
group_id[2] := N'PAL_MASSIVE_PROCESSING_SPECIAL_GROUP_ID';
param_name[2] := N'WEEKLY_SEASONALITY';
int_value[2] := 1;
double_value[2] := NULL;
string_value[2] := NULL;
group_id[3] := N'PAL_MASSIVE_PROCESSING_SPECIAL_GROUP_ID';
param_name[3] := N'DAILY_SEASONALITY';
int_value[3] := 1;
double_value[3] := NULL;
string_value[3] := NULL;
group_id[4] := N'PAL_MASSIVE_PROCESSING_SPECIAL_GROUP_ID';
param_name[4] := N'CHANGEPOINT_PRIOR_SCALE';
int_value[4] := NULL;
double_value[4] := 0.06;
string_value[4] := NULL;
params = UNNEST(:group_id, :param_name, :int_value, :double_value, :string_value);
in_0 = SELECT "STATION_UUID", "PRICE_AT_TIME

In [13]:
amf.model_.head(5).collect()

,GROUP_ID,ROW_INDEX,MODEL_CONTENT
0,11751341-2a8d-4408-af81-a4a01e054314,0,"{""GROWTH"":""linear"",""FLOOR"":0.0,""SEASONALITY_MO..."
1,0e5f8257-c5e6-44f2-8d7f-4ba90f7b9646,0,"{""GROWTH"":""linear"",""FLOOR"":0.0,""SEASONALITY_MO..."
2,0e64e4c6-3d58-44d4-bbe5-377a5a0db2f2,0,"{""GROWTH"":""linear"",""FLOOR"":0.0,""SEASONALITY_MO..."
3,018e8f3e-ae2f-40bc-89c1-bc3fe20eb462,0,"{""GROWTH"":""linear"",""FLOOR"":0.0,""SEASONALITY_MO..."
4,10d13975-7819-4c49-86be-3465bfdc59f9,0,"{""GROWTH"":""linear"",""FLOOR"":0.0,""SEASONALITY_MO..."


In [14]:
result = amf.predict(data=test_rnk_hdf, key="PRICE_AT_TIMESTAMP", group_key="STATION_UUID")

INFO:hana_ml.ml_base:Executing SQL: DO (IN in_1 TABLE ("GROUP_ID" NVARCHAR(100), "ROW_INDEX" INT, "MODEL_CONTENT" NCLOB) => "#PAL_MASSIVE_ADDITIVE_MODEL_FORECAST_MODEL_TBL_0_83FED50E_0B9E_11EE_A990_1ACCC35366B4")
BEGIN
DECLARE group_id NVARCHAR(5000) ARRAY;
DECLARE param_name VARCHAR(5000) ARRAY;
DECLARE int_value INTEGER ARRAY;
DECLARE double_value DOUBLE ARRAY;
DECLARE string_value VARCHAR(5000) ARRAY;
group_id[1] := N'1';
param_name[1] := N'PLACE_HOLDER';
int_value[1] := NULL;
double_value[1] := NULL;
string_value[1] := N'place_holder';
params = UNNEST(:group_id, :param_name, :int_value, :double_value, :string_value);
in_0 = SELECT "STATION_UUID", "PRICE_AT_TIMESTAMP", "E5" FROM (SELECT * FROM "TECHED22"."TEST_RNK_2209") AS "DT_2";
CALL _SYS_AFL.PAL_MASSIVE_ADDITIVE_MODEL_PREDICT(:in_0, :in_1, :params, out_0, out_1, out_2);
CREATE LOCAL TEMPORARY COLUMN TABLE "#PAL_MASSIVE_AMF_PREDICT_RESULT_TBL_0_A698488E_0B9E_11EE_A990_1ACCC35366B4" AS (SELECT * FROM :out_0);
CREATE LOCAL TEMPORAR

In [15]:
# look at predictions for one POS
# predict returns an array of three dataframes. 

result[0].head(10).collect()

,GROUP_ID,PRICE_AT_TIMESTAMP,YHAT,YHAT_LOWER,YHAT_UPPER
0,11751341-2a8d-4408-af81-a4a01e054314,2022-06-14 06:12:08,2.099254,2.050689,2.145868
1,11751341-2a8d-4408-af81-a4a01e054314,2022-06-14 10:31:10,2.038171,1.986486,2.083606
2,11751341-2a8d-4408-af81-a4a01e054314,2022-06-14 11:44:10,2.027534,1.977741,2.074961
3,11751341-2a8d-4408-af81-a4a01e054314,2022-06-14 12:35:09,2.022823,1.970712,2.070847
4,11751341-2a8d-4408-af81-a4a01e054314,2022-06-14 14:09:09,2.018049,1.968824,2.066147
5,11751341-2a8d-4408-af81-a4a01e054314,2022-06-15 06:16:09,2.090663,2.040240,2.138683
6,11751341-2a8d-4408-af81-a4a01e054314,2022-06-15 09:46:15,2.037930,1.989424,2.085616
7,11751341-2a8d-4408-af81-a4a01e054314,2022-06-15 10:34:13,2.029595,1.982141,2.074509
8,11751341-2a8d-4408-af81-a4a01e054314,2022-06-15 14:09:11,2.009389,1.961124,2.055960
9,11751341-2a8d-4408-af81-a4a01e054314,2022-06-15 15:49:09,1.999395,1.951267,2.047700


### Generate HANA artifacts for CAP project

In [16]:
from hana_ml.artifacts.generators import hana
from hana_ml.artifacts.generators.hana import HANAGeneratorForCAP

In [17]:
hanagen = HANAGeneratorForCAP(project_name="PAL_PROPHET_FORECAST_4CAP",
                              output_dir=".",
                              namespace="hana.ml")
hanagen.generate_artifacts(amf, model_position=True, cds_gen=False,tudf=True)